# Analysis: Arrests ANOVA by Region

Workflow: 3 <br>
Goal: Run ANOVA analysis on arrest / population, to compare regions. <br>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import crime_helper as ch

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)

## Initial Setup

In [3]:
ARREST_FILE = 'data/arrest_tkm_state.csv'
EMPLOYMENT_FILE = 'data/employment_state.csv'
OFFENDER_FILE = 'data/offender_tkm_state.csv'
OFFENSE_FILE = 'data/offense_tkm_state.csv'
VICTIM_FILE = 'data/victim_tkm_state.csv'
STATE_FILE = 'data/lookup_state.csv'
REGION_FILE = 'data/lookup_region.csv'

In [4]:
state_df = pd.read_csv(STATE_FILE)
state_df.head()
state_df.info()

,region_code,state_abbr,state_fips_code,state_id,state_name
0,4,AK,2.0,1,Alaska
1,3,AL,1.0,2,Alabama
2,3,AR,5.0,3,Arkansas
3,99,AS,60.0,4,American Samoa
4,4,AZ,4.0,5,Arizona


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 5 columns):
region_code        58 non-null int64
state_abbr         58 non-null object
state_fips_code    57 non-null float64
state_id           58 non-null int64
state_name         58 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 2.3+ KB


In [5]:
region_df = pd.read_csv(REGION_FILE)
region_df

,region_code,region_desc,region_name
0,0,U.S. Territories,U.S. Territories
1,1,Region I,Northeast
2,2,Region II,Midwest
3,3,Region III,South
4,4,Region IV,West
5,99,Other,Other


In [6]:
state_full_df = pd.merge(state_df, region_df, how='inner', on='region_code')
state_full_df.head()

,region_code,state_abbr,state_fips_code,state_id,state_name,region_desc,region_name
0,4,AK,2.0,1,Alaska,Region IV,West
1,4,AZ,4.0,5,Arizona,Region IV,West
2,4,CA,6.0,6,California,Region IV,West
3,4,CO,8.0,7,Colorado,Region IV,West
4,4,HI,15.0,15,Hawaii,Region IV,West


In [7]:
state_clean_df = state_full_df.sort_values('state_abbr').reset_index()
state_clean_df = state_clean_df[['state_abbr', 'state_name', 'region_name']]
state_clean_df.head(10)
state_clean_df.info()

,state_abbr,state_name,region_name
0,AK,Alaska,West
1,AL,Alabama,South
2,AR,Arkansas,South
3,AS,American Samoa,Other
4,AZ,Arizona,West
5,CA,California,West
6,CO,Colorado,West
7,CT,Connecticut,Northeast
8,CZ,Canal Zone,Other
9,DC,District of Columbia,South


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 3 columns):
state_abbr     58 non-null object
state_name     58 non-null object
region_name    58 non-null object
dtypes: object(3)
memory usage: 1.4+ KB


## Load Arrests

In [8]:
arrest_df = pd.read_csv(ARREST_FILE)
arrest_df = arrest_df.rename(columns={'count': 'arrests'})
arrest_df.head()
arrest_df.info()

,state,year,gender,offense,arrests
0,AK,2000,female,aggravated-assault,194
1,AK,2000,female,arson,3
2,AK,2000,female,burglary,49
3,AK,2000,female,curfew,1
4,AK,2000,female,disorderly-conduct,221


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51102 entries, 0 to 51101
Data columns (total 5 columns):
state      51102 non-null object
year       51102 non-null int64
gender     51102 non-null object
offense    51102 non-null object
arrests    51102 non-null int64
dtypes: int64(2), object(3)
memory usage: 1.9+ MB


In [9]:
arrest_total_df = arrest_df.groupby(['state', 'year']).sum().sort_values(['state', 'year'])
arrest_total_df = arrest_total_df.reset_index()
arrest_total_df.head()

,state,year,arrests
0,AK,2000,21354
1,AK,2001,20267
2,AK,2002,19818
3,AK,2003,22746
4,AK,2004,21027


In [10]:
arrest_year_df = arrest_total_df.groupby(['year']).count().sort_values(['year']) 
arrest_year_df = arrest_year_df.rename(columns={'arrests': 'row_count'})
arrest_year_df

,state,row_count
year,,
2000,48,48
2001,50,50
2002,50,50
2003,50,50
2004,50,50
2005,50,50
2006,50,50
2007,50,50
2008,50,50


## Load Employment

In [11]:
employment_df = pd.read_csv(EMPLOYMENT_FILE)
employment_df.head()
employment_df.info()

,agency_ct,civilian_ct,year,female_civilian_ct,female_officer_ct,female_total_ct,male_civilian_ct,male_officer_ct,male_total_ct,pe_ct_per_1000,population,state_abbr,state_name,total_pe_ct
0,36,730.0,2010,474.0,106.0,580.0,256.0,1152.0,1408.0,2.79,711456,AK,Alaska,1988.0
1,35,710.0,2011,472.0,109.0,581.0,238.0,1215.0,1453.0,2.80,725910,AK,Alaska,2034.0
2,35,690.0,2012,453.0,109.0,562.0,237.0,1221.0,1458.0,2.75,735190,AK,Alaska,2020.0
3,33,634.0,2013,443.0,113.0,556.0,191.0,1248.0,1439.0,2.70,739005,AK,Alaska,1995.0
4,33,691.0,2014,456.0,116.0,572.0,235.0,1202.0,1437.0,2.71,740644,AK,Alaska,2009.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 14 columns):
agency_ct             459 non-null int64
civilian_ct           459 non-null float64
year                  459 non-null int64
female_civilian_ct    459 non-null float64
female_officer_ct     459 non-null float64
female_total_ct       459 non-null float64
male_civilian_ct      459 non-null float64
male_officer_ct       459 non-null float64
male_total_ct         459 non-null float64
pe_ct_per_1000        459 non-null float64
population            459 non-null int64
state_abbr            459 non-null object
state_name            459 non-null object
total_pe_ct           459 non-null float64
dtypes: float64(9), int64(3), object(2)
memory usage: 50.3+ KB


In [12]:
population_df = employment_df[['state_abbr', 'year', 'population']]
population_df.head()

,state_abbr,year,population
0,AK,2010,711456
1,AK,2011,725910
2,AK,2012,735190
3,AK,2013,739005
4,AK,2014,740644


In [13]:
population_df.groupby('year').count()

,state_abbr,population
year,,
2010,51,51
2011,51,51
2012,51,51
2013,51,51
2014,51,51
2015,51,51
2016,51,51
2017,51,51
2018,51,51


## Merge DataFrames

In [14]:
merge_df = pd.merge(arrest_total_df, population_df, how='inner', on=['state_abbr', 'year'])
merge_df.head()

KeyError: 'state_abbr'

In [ ]:
merge_df = merge_df[['region_name', 'state', 'year', 'arrests', 'population']]
merge_df.head()
merge_df.shape

In [ ]:
merge_df.region_name.value_counts()

In [ ]:
merge_df.boxplot(column=['arrests', 'offenses', 'victims'])

## Determine how good the data is

In [ ]:
mean_df = merge_df.loc[merge_df['state']=='MD'].mean()
mean_df

In [ ]:
mean_df['arrests']

In [ ]:
check_ls = []
for idx, row in state_df.iterrows():
    state = row['state_abbr']
    
    mean_df = merge_df.loc[merge_df['state']==state].mean()
    std_df = merge_df.loc[merge_df['state']==state].std()
    
    new_dx = {
        'state': state,
        'arrest_mean': round(mean_df['arrests'], 0),
        'arrest_std': round(std_df['arrests'], 0),
        'offense_mean': round(mean_df['offenses'], 0),
        'offense_std': round(std_df['offenses'], 0),
        'victim_mean': round(mean_df['victims'], 0),
        'victim_std': round(std_df['victims'], 0),
        'pop_mean': round(mean_df['population'], 0),
        'pop_std': round(std_df['population'], 0),
    }
    
    check_ls.append(new_dx)

In [ ]:
check_df = pd.DataFrame(check_ls)
check_df.head()

In [ ]:
check_df['stat_arrest'] = check_df['arrest_std'] / check_df['arrest_mean']
check_df['stat_offense'] = check_df['offense_std'] / check_df['offense_mean']
check_df['stat_victim'] = check_df['victim_std'] / check_df['victim_mean']
check_df['stat_pop'] = check_df['pop_std'] / check_df['pop_mean']

check_df = check_df[['state', 'stat_arrest', 'stat_offense', 'stat_victim', 'stat_pop']]
check_df

## Create 3 period Dataframes

In [ ]:
merge_df.groupby(['region_name', 'year']).count()[['state']]

In [ ]:
period_0405_df = combined_df.loc[combined_df['year'].between(2004, 2005)]
period_0405_df.head()
period_0405_df.shape

In [ ]:
period_1112_df = combined_df.loc[combined_df['year'].between(2011, 2012)]
period_1112_df.head()
period_1112_df.shape

In [ ]:
period_1718_df = combined_df.loc[combined_df['year'].between(2017, 2018)]
period_1718_df.head(10)
period_1718_df.shape

In [ ]:
# examine number of counts per region

period_0405_df.groupby(['region_name']).count()[['state', 'year']]
period_1112_df.groupby(['region_name']).count()[['state', 'year']]
period_1718_df.groupby(['region_name']).count()[['state', 'year']]

In [ ]:
save_path = r'data/crime_data_full.csv'
merge_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'